In [2]:
import pandas as pd
import pandas as pd
import spacy
import csv
import os
from collections import Counter

In [14]:
def agregar_weas(df, columna):
    df[columna] = df[columna].apply(lambda x: f'| {x} |')
    return df

In [15]:
datos_dern = pd.read_csv('Src/Corpus/Corpus_Dern.csv')
datos_dern

,Mensaje
0,._.xD
1,Jugamos al rato?
2,Y como descargo más stickers xd
3,Si ese ya lo tengo
4,Pero no hay más?
...,...
12233,a
12234,es que hoy me funan
12235,JAJAJAJAJA
12236,(Rie para no llorar)


In [16]:
datos_esme = pd.read_csv('Src/Corpus/Corpus_Esme.csv')
datos_esme

,Mensaje
0,Hoy me toco tremendo maraton
1,De hecho jajajaj
2,Nooo
3,Niyo
4,Y trngo un bn de tarea
...,...
1561,Cualeeees?
1562,Porfiiii
1563,Yendo
1564,Sacaaaaa


In [17]:
datos_dern=agregar_weas(datos_dern, 'Mensaje')
datos_esme=agregar_weas(datos_esme, 'Mensaje')

In [18]:
datos_esme

,Mensaje
0,| Hoy me toco tremendo maraton |
1,| De hecho jajajaj |
2,| Nooo |
3,| Niyo |
4,| Y trngo un bn de tarea |
...,...
1561,| Cualeeees? |
1562,| Porfiiii |
1563,| Yendo |
1564,| Sacaaaaa |


In [19]:
datos_dern.to_csv('Src/Corpus/Corpus_Dern.csv', index=False)

In [20]:
datos_esme.to_csv('Src/Corpus/Corpus_Esme.csv', index=False)

In [ ]:
def obtener_nombre_archivo(file_name):
    return os.path.splitext(os.path.basename(file_name))[0]

In [3]:
def Generar_Bigramas(corpus):
    nlp = spacy.load('es_core_news_sm')
    nombre_base = obtener_nombre_archivo(corpus.name)

    # Paso 1 tokenizar
    with open(corpus, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        next(reader)
        archivo_tokenizado = f'{nombre_base}_tokenizado.csv'
        with open(archivo_tokenizado, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['Mensaje Tokenizado'])
            for linea in reader:
                doc = nlp(linea[0])
                tokens = [token.text for token in doc]
                writer.writerow([' '.join(tokens)])

    # Paso 2 bigramas
    with open(archivo_tokenizado, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        next(reader)
        archivo_bigramas = f'/Ngramas/{nombre_base}_bigramas.csv'
        with open(archivo_bigramas, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['Bigramas'])
            for linea in reader:
                tokens = linea[0].split()
                bigrams = [(tokens[i], tokens[i + 1]) for i in range(len(tokens) - 1)]  # Crear bigramas
                for bigram in bigrams:
                    writer.writerow([' '.join(bigram)])

    # Paso 3 frecuencia y probabilidad conjunta
    with open(archivo_bigramas, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        next(reader)
        bigramas = []
        tokens_bi = []
        for row in reader:
            bigrama = row[0]
            bigramas.append(bigrama)
            tokens_bi.append(row[0].split())
        freq_bigramas = Counter(bigramas)
        freq_total_bigramas = len(tokens_bi)
    
    archivo_frecuencias_bigramas = f'/Frecuencias/{nombre_base}_frecuencia_bigrama.csv'
    with open(archivo_frecuencias_bigramas, 'w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["Bigrama","Frecuencia","Frecuencia Relativa"])
        for bigrama, freq in freq_bigramas.items():
            writer.writerow([bigrama, freq, freq / freq_total_bigramas])

    #Paso 4 juntar toas las weas
    archivo_frecuencias_proba_bigramas = f'/Frecuencias/{nombre_base}_frecuencia_proba_bigrama.csv'
    with open(archivo_frecuencias_proba_bigramas, 'w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["Bigrama","Frecuencia","Probabilidad Conjunta"])
        for bigrama, freq in freq_bigramas.items():
            proba = freq / freq_total_bigramas
            proba = round(proba, 6)
            writer.writerow([bigrama, freq, proba])

    archivo_bigrmas_final = f'/Ngramas/{nombre_base}_bigramas_final.csv'
    with open(archivo_frecuencias_proba_bigramas, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        next(reader)
        datos_bigramas = []
        for row in reader:
            datos_bigramas.append(row)
    new_data = []
    terms = [term for row in datos_bigramas for term in row[0].split()]
    frecuencia_terms = Counter(terms)
    new_data.append(["Term 1", "Term 2", "Frequency of Bigram", "Frecuency of Context", "Conditional Probability of Bigram"])

    for row in datos_bigramas:
        bigrama = row[0]
        frecuencia_bigrama = int(row[1])
        probabilidad_conjunta = float(row[2])
        term1, term2 = bigrama.split()
        frecuencia_term = frecuencia_terms[term1]

        new_data.append([term1, term2, frecuencia_bigrama, frecuencia_term, probabilidad_conjunta])

    new_data_sorted = sorted(new_data[1:], key=lambda x: x[2], reverse=True)
    new_data_sorted.insert(0, new_data[0])

    with open(archivo_bigrmas_final, 'w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(new_data_sorted)

In [ ]:
def Generar_Trigramas(corpus):
    pass